In [1]:
import MetaTrader5 as mt5 
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import adfuller 
from datetime import datetime
import requests
import pandas as pd
import numpy as np
import warnings
import json
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

def get_rates(pair1, timeframe, x):
    pair1 = pd.DataFrame(mt5.copy_rates_from_pos(pair1, timeframe, 0, x))
    pair1['time'] = pd.to_datetime(pair1['time'], unit = 's')
    return pair1[['time','open', 'high', 'low', 'close']].set_index('time')
def calc_stats(df):

    stats_df = pd.DataFrame()
    stats_df['mean'] = df.mean()
    stats_df['std'] = df.std()
    # print('calculated')
    return stats_df

# BIS

## Data Collection

In [982]:
combined_dfs_m = {}
for country in count_codes:

    # Define the endpoint parameters
    flow = 'BIS,WS_EER_M,1.0'  # Example: Version 1.0 of the WS_EER_M domain, maintained by the BIS
    key = f'M.N.N.{country[1]}'
    start_period = '2000'  # Example: Start year 2000
    end_period = '2024'  # Example: End year 2020
    detail = 'full'  # Example: All data and documentation

    # Construct the endpoint URL
    endpoint_url = f'{base_url}/data/{flow}/{key}/all'

    # Define the query parameters
    query_params = {
        'startPeriod': start_period,
        'endPeriod': end_period,
        'detail': detail
    }

    # Make the GET request
    response = requests.get(endpoint_url, params=query_params)

    # Check for a successful response
    if response.status_code == 200:
        # Assign the text of the response to xml_data
        xml_data = response.text

        # Parse the XML data
        root = ET.fromstring(xml_data)

        # Initialize empty lists to store the data
        time_periods = []
        obs_values = []

        # Iterate through the XML and extract the desired information
        for obs in root.findall(".//Obs"):
            time_period = obs.get('TIME_PERIOD')
            obs_value = obs.get('OBS_VALUE')
            time_periods.append(time_period)
            obs_values.append(obs_value)

        # Create a DataFrame
        df = pd.DataFrame({
            'Time_Period': time_periods,
            'OBS_Value': obs_values
        })

        df['OBS_Value'] = df['OBS_Value'].replace('NaN', np.nan)
        # Drop rows with NaN values
        df.dropna(subset=['OBS_Value'], inplace=True)
        df['OBS_Value'] = df['OBS_Value'].astype(float)
        df['Time_Period'] = pd.to_datetime(df['Time_Period'])
        
        df = df.set_index('Time_Period')
        print(f"Getting {country[0]}'s rates")
        rates = get_rates(country[0], mt5.TIMEFRAME_D1, 2500)
        
        combined = pd.concat([df[-len(rates):], rates['close']], join = 'outer', axis = 1)
        
        combined_dfs_m[country[1]] = combined.dropna()

    else:
        print(f'Failed to retrieve data: {response.status_code}')

Getting AUDUSD.a's rates
Getting USDCAD.a's rates
Getting USDCHF.a's rates
Getting GBPUSD.a's rates
Getting USDJPY.a's rates
Getting NZDUSD.a's rates
Getting EURUSD.a's rates


In [989]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd

combined_dfs = {}

# Define the base URL for the BIS Stats API
base_url = 'https://stats.bis.org/api/v1'

count_codes = [['AUDUSD.a','AU'], ['USDCAD.a', 'CA'],
               ['USDCHF.a','CH'], ['GBPUSD.a', 'GB'],
               ['USDJPY.a', 'JP'], ['NZDUSD.a', 'NZ'], 
               ['EURUSD.a', 'XM']]

for country in count_codes:

    # Define the endpoint parameters
    flow = 'BIS,WS_EER_D,1.0'  # Example: Version 1.0 of the WS_EER_M domain, maintained by the BIS
    key = f'D.N.N.{country[1]}'
    start_period = '2000'  # Example: Start year 2000
    end_period = '2025'  # Example: End year 2020
    detail = 'full'  # Example: All data and documentation

    # Construct the endpoint URL
    endpoint_url = f'{base_url}/data/{flow}/{key}/all'

    # Define the query parameters
    query_params = {
        'startPeriod': start_period,
        'endPeriod': end_period,
        'detail': detail
    }

    # Make the GET request
    response = requests.get(endpoint_url, params=query_params)

    # Check for a successful response
    if response.status_code == 200:
        # Assign the text of the response to xml_data
        xml_data = response.text

        # Parse the XML data
        root = ET.fromstring(xml_data)

        # Initialize empty lists to store the data
        time_periods = []
        obs_values = []

        # Iterate through the XML and extract the desired information
        for obs in root.findall(".//Obs"):
            time_period = obs.get('TIME_PERIOD')
            obs_value = obs.get('OBS_VALUE')
            time_periods.append(time_period)
            obs_values.append(obs_value)

        # Create a DataFrame
        df = pd.DataFrame({
            'Time_Period': time_periods,
            'OBS_Value': obs_values
        })

        df['OBS_Value'] = df['OBS_Value'].replace('NaN', np.nan)
        # Drop rows with NaN values
        df.dropna(subset=['OBS_Value'], inplace=True)
        df['OBS_Value'] = df['OBS_Value'].astype(float)
        df['Time_Period'] = pd.to_datetime(df['Time_Period'])
        
        df = df.set_index('Time_Period')
        print(f"Getting {country[0]}'s rates")
        rates = get_rates(country[0], mt5.TIMEFRAME_D1, 2500)
        
        combined = pd.concat([df[-len(rates):], rates['close']], join = 'outer', axis = 1)
        
        combined_dfs[country[1]] = combined.dropna()

    else:
        print(f'Failed to retrieve data: {response.status_code}')

Getting AUDUSD.a's rates
Getting USDCAD.a's rates
Getting USDCHF.a's rates
Getting GBPUSD.a's rates
Getting USDJPY.a's rates
Getting NZDUSD.a's rates
Getting EURUSD.a's rates


In [992]:
# Compute Expected Returns
weekly_dfs = {}
features = ['OBS_Value', 'close']

for name, dfs in combined_dfs.items():
    df = combined_dfs[name]
    # Assuming 'df' is your DataFrame
    df['date'] = pd.to_datetime(df.index)
    df.set_index('date', inplace=True)

    # Resample to get the last value of each week
    weekly = df.resample('W').last()
    weekly_dfs[name] = weekly
    
    # Add a new column for Monthly OBS Values in the weekly DataFrame
    weekly_dfs[name]['Monthly_OBS_Value'] = np.nan

    # Iterate through the monthly DataFrame
    for date, row in combined_dfs_m[name].iterrows():
        # Find all weekly dates that are in the same month and year as the monthly date
        mask = (weekly_dfs[name].index.month == date.month) & (weekly_dfs[name].index.year == date.year)
        weekly_dfs[name].loc[mask, 'Monthly_OBS_Value'] = row['OBS_Value']

    # Forward fill the NaN values in the Monthly_OBS_Value column
    weekly_dfs[name]['Monthly_OBS_Value'] = weekly_dfs[name]['Monthly_OBS_Value'].ffill()
    
for df in weekly_dfs.values():
    df['W_EER_ret'] = df['OBS_Value'].pct_change()
    df['M_EER_ret'] = df['Monthly_OBS_Value'].pct_change()
    df['close_ret'] = df['close'].pct_change()
    df['ret_diff'] = df['close_ret'] - df['W_EER_ret']

In [994]:
last_update = {}

for name, df in weekly_dfs.items():
    last_update[name] = df['W_EER_ret'].iloc[-1]

In [995]:
last_update.keys()
symbols = ['AUDUSD.a', 'USDCAD.a', 'USDCHF.a', 'GBPUSD.a', 'USDJPY.a', 'NZDUSD.a', 'EURUSD.a']

last_update = {symbols[i]: value for i, (key, value) in enumerate(last_update.items())}

## Linear Regression Model

In [996]:
stats = {}
models = {}
best_model_vars_dict = {}

X_vars = [['W_EER_ret', 'M_EER_ret'], ['W_EER_ret']]
for df in weekly_dfs:
    best_score = float('-inf')  # Initialize the best score as negative infinity
    best_model = None  # Initialize the best model
    for variables in X_vars:
        print(f" --------- Testing variable list {variables} --------- ")
        print(f"Looping through {df}")
        X = weekly_dfs[df][variables].dropna()
        y = weekly_dfs[df]['close_ret'].dropna()
        y = y.iloc[-len(X):]

        # Splitting the dataset into training and testing sets
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

        # Creating the linear regression model
        model = LinearRegression()
        
        # Training the model
        model.fit(X_train, y_train)

        # Making predictions using the testing set
        y_pred = model.predict(X_test)

        # The coefficients
        # print('Coefficients:', model.coef_)
        # The mean squared error
        # print('Mean squared error (MSE): %.6f' % mean_squared_error(y_test, y_pred))
        # The coefficient of determination: 1 is perfect prediction
        print('Coefficient of determination (R^2): %.6f' % r2_score(y_test, y_pred))
        new_score = r2_score(y_test, y_pred)
        # Check if this model is better
        if new_score > best_score:
            print(f"**Model with {variables} performs better. Updating best model.**")
            best_score = new_score
            best_model = model
            best_model_vars = variables
            
    models[df] = best_model
    best_model_vars_dict[df] = best_model_vars  # Store the best variables in the dictionary

    stats[df] = calc_stats(weekly_dfs[df][best_model_vars])

 --------- Testing variable list ['W_EER_ret', 'M_EER_ret'] --------- 
Looping through AU
Coefficient of determination (R^2): 0.519629
**Model with ['W_EER_ret', 'M_EER_ret'] performs better. Updating best model.**
 --------- Testing variable list ['W_EER_ret'] --------- 
Looping through AU
Coefficient of determination (R^2): 0.734967
**Model with ['W_EER_ret'] performs better. Updating best model.**
 --------- Testing variable list ['W_EER_ret', 'M_EER_ret'] --------- 
Looping through CA
Coefficient of determination (R^2): 0.581836
**Model with ['W_EER_ret', 'M_EER_ret'] performs better. Updating best model.**
 --------- Testing variable list ['W_EER_ret'] --------- 
Looping through CA
Coefficient of determination (R^2): 0.717895
**Model with ['W_EER_ret'] performs better. Updating best model.**
 --------- Testing variable list ['W_EER_ret', 'M_EER_ret'] --------- 
Looping through CH
Coefficient of determination (R^2): 0.214635
**Model with ['W_EER_ret', 'M_EER_ret'] performs better. 

In [998]:
all_returns = pd.DataFrame()
for sym in count_codes:
    # print(f'Iterating thru {sym}')
    rates = get_rates(sym[0], mt5.TIMEFRAME_D1, 2000)
    rates = rates['close']
    rates = rates.rename(f'{sym[0]}_close')
    all_returns = pd.concat([all_returns, rates], axis = 1)
latest_prices = all_returns.iloc[-1]

## MC Simulation

In [46]:
import numpy as np
from pypfopt import EfficientFrontier, risk_models, expected_returns
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices 

In [143]:
def scale_weights(weights):

    total_weights = __builtins__.sum(weights.values)
    
    scaled = {currency: weight / total_weights * 100 for currency, weight in weights.items()}

    if isinstance(weights, pd.Series):
        return pd.Series(scaled)
    return scaled

In [103]:
def MC_Sim(simulations, portfolio_val):

    all_weights = []

    for i in range(simulations):

        if (i + 1) % 250 == 0:
            print(f'{i + 1} simulations ran')

        expected_return_lst = {}
        for df, values in stats.items():
            mean = values['mean']
            std = values['std']

            for df2, lr_model in models.items():
                if df == df2:
                    num_features = len(best_model_vars_dict[df2])  # Number of features the model was trained on
                    # Ensure mean and std have correct dimensions
                    mean_values = np.full(num_features, mean)
                    std_values = np.full(num_features, std)
                    # Generate random values for each feature
                    random_eer_ret = np.random.normal(mean_values, std_values, (1, num_features))
                    # Predict using the model
                    expected_return_lst[df2] = lr_model.predict(random_eer_ret)

        lst = []
        for i in expected_return_lst.values():
            lst.append(i)

        highest_return = max(lst)
        desired_scale = 0.05  # 5%
        scaling_factor = desired_scale / highest_return

        # Apply the scaling factor
        scaled_returns = [ret * scaling_factor for ret in lst] 
        scaled_values = [x[0] * 100 for x in scaled_returns]
        fin = []
        for i in scaled_values:
            fin.append(i) # changed from ABS # 

        # Calculate expected returns and the covariance matrix
        mu = fin
        S = risk_models.sample_cov(all_returns)

        # Optimize for the minimised volatility
        ef = EfficientFrontier(mu, S, weight_bounds = (-0.25, 0.25)) ## Changed weight bounds to reflect a portfolio with shorting
        weights = ef.efficient_risk(0.025, market_neutral = False) ## Switched from True to False to reflect shorting
        cleaned_weights = ef.clean_weights()

        exp_ret, exp_vol, sharpe = ef.portfolio_performance()
        da = DiscreteAllocation(
        cleaned_weights, latest_prices, 
        total_portfolio_value = portfolio_val, short_ratio = None)

        # Perform the discrete allocation
        allocation, leftover = da.greedy_portfolio()
        # Calculate the total investment in each asset
        total_investment = {ticker: allocation[ticker] * latest_prices[ticker] for ticker in allocation}

        # Total value of the portfolio
        total_portfolio_value = sum(total_investment.values()) + leftover

        # Convert to percentages
        portfolio_percentages = {ticker: (value / total_portfolio_value) * 100 for ticker, value in total_investment.items()}

        # Store the weights from each simulation
        # all_weights[i, :] = np.array(list(portfolio_percentages.values()))
        all_weights.append([portfolio_percentages, exp_ret, exp_vol])
        
    return all_weights

In [1003]:
all_weights = MC_Sim(8000, 3000)

250 simulations ran
500 simulations ran
750 simulations ran
1000 simulations ran
1250 simulations ran
1500 simulations ran
1750 simulations ran
2000 simulations ran
2250 simulations ran
2500 simulations ran
2750 simulations ran
3000 simulations ran
3250 simulations ran
3500 simulations ran
3750 simulations ran
4000 simulations ran
4250 simulations ran
4500 simulations ran
4750 simulations ran
5000 simulations ran
5250 simulations ran
5500 simulations ran
5750 simulations ran
6000 simulations ran
6250 simulations ran
6500 simulations ran
6750 simulations ran
7000 simulations ran
7250 simulations ran
7500 simulations ran
7750 simulations ran
8000 simulations ran


In [1004]:
weights_list = []
returns_list = []
volatility_list = []

# Extract values into separate lists
for item in all_weights:
    weights_list.append(item[0])  # Portfolio weights dictionary
    returns_list.append(item[1])  # Expected return
    volatility_list.append(item[2])  # Expected volatility

# Convert weights to DataFrame for easier mean calculation
df_weights = pd.DataFrame(weights_list)

# Calculate mean weights, returns, and volatility
mean_weights = df_weights.mean()
mean_return = np.mean(returns_list)
mean_volatility = np.mean(volatility_list)

print("Mean Portfolio Weights:")
print(mean_weights)
print("\nMean Expected Return:", mean_return)
print("Mean Expected Volatility:", mean_volatility)

Mean Portfolio Weights:
AUDUSD.a_close     7.536998
USDCAD.a_close    23.623394
USDCHF.a_close    21.525681
EURUSD.a_close    20.959173
GBPUSD.a_close     6.498291
NZDUSD.a_close     7.029920
USDJPY.a_close    12.516694
dtype: float64

Mean Expected Return: 2.322274875279652
Mean Expected Volatility: 0.025000001174502975


In [1005]:

weights_list = []
returns_list = []
volatility_list = []

# Extract values into separate lists
for item in all_weights:
    weights_list.append(item[0])  # Portfolio weights dictionary
    returns_list.append(item[1])  # Expected return
    volatility_list.append(item[2])  # Expected volatility

# Convert weights to DataFrame for easier mean and IQR calculation
df_weights = pd.DataFrame(weights_list)

# Calculate mean weights and IQR for each asset
mean_weights = df_weights.mean()
iqr_weights = df_weights.quantile(0.75) - df_weights.quantile(0.25)
iqr_25_weights = df_weights.quantile(0.25)
iqr_75_weights = df_weights.quantile(0.75)

# Combine return and volatility data
df_performance = pd.DataFrame({
    'expected_return': returns_list,
    'expected_volatility': volatility_list
})

# Calculate mean return and volatility
mean_return = df_performance['expected_return'].mean()
iqr_25_ret = df_performance['expected_return'].quantile(0.25)
iqr_75_ret = df_performance['expected_return'].quantile(0.75)

mean_volatility = df_performance['expected_volatility'].mean()
iqr_25_vol = df_performance['expected_volatility'].quantile(0.25)
iqr_75_vol = df_performance['expected_volatility'].quantile(0.75)

iqr_25_dict = {}
iqr_75_dict = {}
mean_dict = {}

for index, row in df_weights.iterrows():
    row_sum = row.sum()
    shortfall = 100 - row_sum
    null_columns = row[row.isnull()].index  # Identify columns with NaN values
    df_weights.loc[index, null_columns] = shortfall / len(null_columns)  # Distribute the shortfall evenly

In [1006]:
iqr_25_dict['weights'] = iqr_25_weights
iqr_75_dict['weights'] = iqr_75_weights
mean_dict['weights'] = mean_weights

iqr_25_dict['ret'] = iqr_25_ret
iqr_75_dict['ret'] = iqr_75_ret
mean_dict['ret'] = mean_return

iqr_25_dict['vol'] = iqr_25_vol
iqr_75_dict['vol'] = iqr_75_vol
mean_dict['vol'] = mean_volatility

In [1007]:
weights = {}
all_dicts = {}

all_dicts['iqr25'] = iqr_25_dict
all_dicts['iqr75'] = iqr_75_dict
all_dicts['mean_dict'] = mean_dict
highest_ret = 0

for df, items in all_dicts.items():

    print(f"Iterating through {df}")
    if all_dicts[df]['ret'] > highest_ret:
        highest_ret = all_dicts[df]['ret']
        weights = all_dicts[df]['weights']
        print(f"{df} has the highest return at {highest_ret} with weights"
              f"\n{weights}")

Iterating through iqr25
iqr25 has the highest return at 0.9967296576728408 with weights
AUDUSD.a_close    -7.504688
USDCAD.a_close    23.688593
USDCHF.a_close    17.768128
EURUSD.a_close    17.745703
GBPUSD.a_close    -1.199485
NZDUSD.a_close    -7.256452
USDJPY.a_close     6.133766
Name: 0.25, dtype: float64
Iterating through iqr75
iqr75 has the highest return at 2.7450053786788553 with weights
AUDUSD.a_close    25.140372
USDCAD.a_close    25.447142
USDCHF.a_close    25.511766
EURUSD.a_close    25.547300
GBPUSD.a_close    15.465178
NZDUSD.a_close    25.069856
USDJPY.a_close    16.598245
Name: 0.75, dtype: float64
Iterating through mean_dict


# Lot Size Allocation and Order Functions

In [349]:
def calculate_lot_sizes(monetary_allocation, latest_prices_dict, lot_size_unit):
    lot_sizes = {}
    
    latest_prices_dict = dict(latest_prices_dict)
    
    for currency_pair, allocation in monetary_allocation.items():
        # Extract the latest price for the current currency pair
        val = latest_prices_dict[currency_pair]

        # Determine pip value based on the currency pair
        if currency_pair[:3] == 'USD':
            # USD is the base currency
            if 'JPY' in currency_pair[3:]:  # JPY exception
                pip_value = 10 / val * 100
            else:
                pip_value = 10 / val
        else:
            # USD is the quote currency or pair does not involve USD
            pip_value = 10

        # print(f"Pip value for {currency_pair} is {pip_value}. Now calculating lots...")

        # Standard lot value calculation (100,000 units)
        standard_lot_value = 100000 * pip_value

        # Calculate lot size
        lot_size = allocation / standard_lot_value

        # Convert to desired lot size unit
        if lot_size_unit == 'micro':
            # 1 micro lot = 1/100th of a standard lot
            lot_size *= 100
        elif lot_size_unit == 'mini':
            # 1 mini lot = 1/10th of a standard lot
            lot_size *= 10

        lot_sizes[currency_pair] = round(lot_size, 2)  # Rounding to 2 decimal places

    return lot_sizes

In [971]:
def eer_os(last_update, lot_sizes, comment):
    
    for sym, lotsize in lot_sizes.items():

        for symbol, value in last_update.items():

            if sym[:8] == symbol:

                if symbol[:3] == 'USD':
                    print(f"Looping through {symbol[:6]}")
                    if value > 0:
                        send_order(symbol[:6], 'sell', lotsize, comment)
                        print(f"Selling {symbol[:6]}")
                    else:
                        pass
                        send_order(symbol[:6], 'buy', lotsize, comment)
                else:
                    print(f"Looping through {symbol[:6]}")
                    if value > 0:
                        print(f"Buying {symbol[:6]}")
                        send_order(symbol[:6], 'buy', lotsize, comment)
                    else:
                        print(f"Selling {symbol[:6]}")
                        send_order(symbol[:6], 'sell', lotsize, comment)
                        
def close_position(position, comment):
    
    tick = mt5.symbol_info_tick(position.symbol)

    request = {
        "action" : mt5.TRADE_ACTION_DEAL,
        "position": position.ticket,
        "symbol": position.symbol,
        "volume": position.volume,
        "type": mt5.ORDER_TYPE_BUY if position.type == 1 else mt5.ORDER_TYPE_SELL,
        "price": tick.ask if position.type == 1 else tick.bid,
        "deviation": 20,
        "magic": 100,
        "comment": comment,
        'type_time': mt5.ORDER_TIME_GTC,
        'type_filling':mt5.ORDER_FILLING_IOC,

        }
    result = mt5.order_send(request)
    
def send_order(symbol, side, lot, comment):
    
    if side.lower() == 'sell':
        order_type = mt5.ORDER_TYPE_SELL
        price = mt5.symbol_info_tick(symbol).bid
    elif side.lower() == 'buy':
        order_type = mt5.ORDER_TYPE_BUY
        price = mt5.symbol_info_tick(symbol).ask
    
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": lot,
        "type": order_type,
        "price": price,
        "deviation": 5,
        "magic": 234000,
        "comment": comment,
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC,
    }
    result = mt5.order_send(request)

def close_all():
    close_positions = []
    open_positions = mt5.positions_get()
    open_positions
    for i in open_positions:
        close_positions.append(i)
        
    for pos in close_positions:
        close_position(pos)

# COT report

## Data / Functions

In [623]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm
import numpy as np
import cot_reports as cot
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, mean_squared_error

def get_rates(pair1, timeframe, x):
    pair1 = pd.DataFrame(mt5.copy_rates_from_pos(pair1, timeframe, 0, x))
    pair1['time'] = pd.to_datetime(pair1['time'], unit = 's')
    return pair1[['time','open', 'high', 'low', 'close']].set_index('time')
def get_rates_w(pair1, timeframe, x):
    
    pair1 = pd.DataFrame(mt5.copy_rates_from_pos(pair1, timeframe, 0, x))
    
    pair1['time'] = pd.to_datetime(pair1['time'], unit='s')
    pair1 = pair1[['time', 'open', 'high', 'low', 'close']].set_index('time')
    
    # Resample to weekly data starting from the day after COT report release
    weekly_data = pair1.resample('W-TUE', label='left').agg({
        'open': 'first',
        'high': 'max',
        'low': 'min',
        'close': 'last'
    })

    return weekly_data

# Define the parameter grid
param_grid_rf = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, 30, None],
    'min_samples_split': [2, 5, 10]
}

def rf_training():
    rf = RandomForestRegressor(random_state=42)

    # Define the scoring function
    mse_scorer = make_scorer(mean_squared_error, greater_is_better=False)

    # Instantiate the grid search model
    grid_search_rf = GridSearchCV(estimator=rf, param_grid=param_grid_rf, 
                                  cv=3, n_jobs=-1, scoring=mse_scorer, verbose=2)

    # Fit the grid search to the data
    grid_search_rf.fit(X_train, y_train)

    # Best parameters
    print("Best parameters for Random Forest:", grid_search_rf.best_params_)

    rf_best = grid_search_rf.best_params_
    
    return rf_best

sym_lst = ['SWISS FRANC', 'BRITISH POUND STERLING', 'JAPANESE YEN', 'EURO FX', 'NEW ZEALAND DOLLAR', 'AUSTRALIAN DOLLAR', 
       'EURO FX/JAPANESE YEN XRATE']

In [625]:
df = pd.DataFrame()
begin_year = 2015
end_year = 2023
for i in range(begin_year, end_year + 1):
    single_year = pd.DataFrame(cot.cot_year(i, cot_report_type='legacy_futopt')) 
    df = pd.concat([df, single_year], axis = 0)

t1 = []
t2 = []
temp = df.set_index('Market and Exchange Names').T
for col in temp.columns.unique():
    if 'CHICAGO MERCANTILE EXCHANGE' in col:
        t1.append(col)

Selected: legacy_futopt
Downloaded single year data from: 2015
Selected: legacy_futopt
Downloaded single year data from: 2016
Selected: legacy_futopt
Downloaded single year data from: 2017
Selected: legacy_futopt
Downloaded single year data from: 2018
Selected: legacy_futopt
Downloaded single year data from: 2019
Selected: legacy_futopt
Downloaded single year data from: 2020
Selected: legacy_futopt
Downloaded single year data from: 2021
Selected: legacy_futopt
Downloaded single year data from: 2022
Selected: legacy_futopt
Downloaded single year data from: 2023


In [633]:
t1 = []
t2 = []
temp = df.set_index('Market and Exchange Names').T
for col in temp.columns.unique():
    if 'CHICAGO MERCANTILE EXCHANGE' in col:
        t1.append(col)
        
for sym in sym_lst:
    for instrument in t1:
        if sym in instrument:
            # print(f"match found on {sym} and {instrument}")
            t2.append(instrument)
selected_cols = temp[t2]

In [634]:
fields_to_drop = [
    "Contract Units",
    "CFTC Contract Market Code (Quotes)",
    "CFTC Market Code in Initials (Quotes)",
    "CFTC Commodity Code (Quotes)",
    "CFTC Contract Market Code",
    "CFTC Market Code in Initials",
    "CFTC Region Code",
    "CFTC Commodity Code", 
    
]

df_dropped = selected_cols.T.drop(columns = fields_to_drop, errors = 'ignore')
df_dict = dict(df_dropped.T)

for name, df in df_dict.items():
    # df[name]
    df = df_dict[name]
    df = df_dict[name].T.reset_index()
    df = df.rename(columns = {'As of Date in Form YYYY-MM-DD' : 'Date'})
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace = True)
    df = df.drop(columns = ['Market and Exchange Names', 'As of Date in Form YYMMDD'])
    df_dict[name] = df
    df_dict[name].sort_index(inplace = True)

In [683]:
cleaned_list = ['SWISS FRANC',
 'BRITISH POUND STERLING',
 'JAPANESE YEN',
 'EURO FX/JAPANESE YEN XRATE',
 'EURO FX',
 'NEW ZEALAND DOLLAR',
 'AUSTRALIAN DOLLAR']

sym_rates = ['USDCHF.a', 'GBPUSD.a', 'USDJPY.a', 'EURJPY.a', 'EURUSD.a', 'NZDUSD.a', 'AUDUSD.a']

In [684]:
new_cot_dict = dict(zip(cleaned_list, df_dict.values()))

In [685]:
combined_info = dict(zip(cleaned_list, sym_rates))

In [722]:
rf_results = {}
for cot_data, sym in combined_info.items():
    print(f" ------- Iterating through {sym} -------")
    
    cot_data = new_cot_dict[cot_data]
    
    # Convert all columns to numeric, replacing non-numeric values with NaN
    for col in cot_data.columns:
        cot_data[col] = pd.to_numeric(cot_data[col], errors='coerce')

    # Iterate through columns and drop those with a sum of 0
    cols_to_drop = [col for col in cot_data.columns if cot_data[col].sum() == 0]
    cot_data.drop(columns=cols_to_drop, inplace=True)
    
    X = cot_data[['Open Interest (All)', 'Noncommercial Positions-Spreading (All)', 'Commercial Positions-Long (All)', 
                           'Commercial Positions-Short (All)', ' Total Reportable Positions-Long (All)', 'Total Reportable Positions-Short (All)',
               'Traders-Noncommercial-Spreading (All)']]
    X = X.dropna()
    
    rate = get_rates_w(sym, mt5.TIMEFRAME_D1, (len(X) * 5))
    rate['close_lag1'] = rate['close'].shift(1)

    y = rate['close_lag1']
    length_diff = len(y) - len(X)
    if length_diff != 0:
        
        y = y.iloc[length_diff:]
    else:
        pass

        # Splitting the dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

    rf_best = rf_training()
    # Create the Random Forest model
    model = RandomForestRegressor(**rf_best)  # You can tune n_estimators and other parameters
    
    # Training the model
    model.fit(X_train, y_train)

    # Making predictions using the testing set
    y_pred = model.predict(X_test)

    # The coefficients
    # print('Coefficients:', model.coef_)
    # The mean squared error
    print('Mean squared error (MSE): %.6f' % mean_squared_error(y_test, y_pred))
    # The coefficient of determination: 1 is perfect prediction
    print('Coefficient of determination (R^2): %.6f' % r2_score(y_test, y_pred))
    
    rf_results[sym] = [r2_score(y_test, y_pred), model]

 ------- Iterating through USDCHF.a -------
Fitting 3 folds for each of 36 candidates, totalling 108 fits
Best parameters for Random Forest: {'max_depth': 10, 'min_samples_split': 2, 'n_estimators': 300}
Mean squared error (MSE): 0.000526
Coefficient of determination (R^2): 0.667727
 ------- Iterating through GBPUSD.a -------
Fitting 3 folds for each of 36 candidates, totalling 108 fits
Best parameters for Random Forest: {'max_depth': 20, 'min_samples_split': 2, 'n_estimators': 200}
Mean squared error (MSE): 0.002557
Coefficient of determination (R^2): 0.393156
 ------- Iterating through USDJPY.a -------
Fitting 3 folds for each of 36 candidates, totalling 108 fits
Best parameters for Random Forest: {'max_depth': 30, 'min_samples_split': 2, 'n_estimators': 200}
Mean squared error (MSE): 57.592008
Coefficient of determination (R^2): 0.720441
 ------- Iterating through EURJPY.a -------
Fitting 3 folds for each of 36 candidates, totalling 108 fits
Best parameters for Random Forest: {'max_

In [682]:
optimised_dict = {}
for key in lin_results:
    if key in rf_results:
        # Compare R² scores and choose the higher one
        if lin_results[key][0] > rf_results[key][0]:
            optimised_dict[key] = lin_results[key]
        else:
            optimised_dict[key] = rf_results[key]

optimised_dict

{'USDCHF.a': [0.6704273127407699,
  RandomForestRegressor(max_depth=10, n_estimators=300)],
 'GBPUSD.a': [0.3699800637740899,
  RandomForestRegressor(max_depth=20, n_estimators=200)],
 'USDJPY.a': [0.7179412430517143,
  RandomForestRegressor(max_depth=30, n_estimators=200)],
 'EURJPY.a': [0.9586310372275183,
  RandomForestRegressor(max_depth=30, n_estimators=300)],
 'EURUSD.a': [0.7802222520268962,
  RandomForestRegressor(max_depth=20, n_estimators=300)],
 'NZDUSD.a': [0.811431227489253,
  RandomForestRegressor(max_depth=10, n_estimators=200)],
 'AUDUSD.a': [0.7134857444539408,
  RandomForestRegressor(max_depth=30, n_estimators=200)]}

## Predicting

In [834]:
forecasts = {}
for cot_data, sym in combined_info.items():
    
    print(f" ------- Iterating through {sym} -------")
    
    cot_data = new_cot_dict[cot_data]
    
    # Convert all columns to numeric, replacing non-numeric values with NaN
    for col in cot_data.columns:
        cot_data[col] = pd.to_numeric(cot_data[col], errors='coerce')

    # Iterate through columns and drop those with a sum of 0
    cols_to_drop = [col for col in cot_data.columns if cot_data[col].sum() == 0]
    cot_data.drop(columns=cols_to_drop, inplace=True)
    
    features = cot_data[['Open Interest (All)', 'Noncommercial Positions-Spreading (All)', 'Commercial Positions-Long (All)', 
                           'Commercial Positions-Short (All)', ' Total Reportable Positions-Long (All)', 'Total Reportable Positions-Short (All)',
                           'Traders-Noncommercial-Spreading (All)']]
    
    features = features.iloc[[-1]]  # Use double brackets to keep DataFrame structure
    
    for pair, model in optimised_dict.items():
        if sym == pair:
            prediction_model = model[1]
            forecast = prediction_model.predict(features)  # features is now a DataFrame with correct shape
            print(forecast)
            forecasts[sym] = float(forecast)

 ------- Iterating through USDCHF.a -------
[0.88099279]
 ------- Iterating through GBPUSD.a -------
[1.2746551]
 ------- Iterating through USDJPY.a -------
[139.858915]
 ------- Iterating through EURJPY.a -------
[157.35320829]
 ------- Iterating through EURUSD.a -------
[1.09800097]
 ------- Iterating through NZDUSD.a -------
[0.64023244]
 ------- Iterating through AUDUSD.a -------
[0.69976125]


In [852]:
current_prices = {}

for sym in sym_rates:
    rate = get_rates(sym, mt5.TIMEFRAME_D1, 1)
    current_prices[sym] = rate['close']

In [918]:
cot_positions = {}

for sym, forecast in forecasts.items():
    for pair, cur_price in current_prices.items():
        if sym == pair:
            print(forecast, cur_price[0])
            cot_positions[sym] = forecast - cur_price[0] 

0.8809927934351901 0.84936
1.274655099999999 1.264
139.85891500000008 141.935
157.35320828571403 155.581
1.0980009666666657 1.0961400000000001
0.6402324383333337 0.62694
0.6997612499999992 0.67652


# Funding Pips Accounts

In [1039]:
last_update

{'AUDUSD.a': 0.004129850172877481,
 'USDCAD.a': -0.0013612056392805494,
 'USDCHF.a': -0.006891205099491704,
 'GBPUSD.a': 0.0013651877133105117,
 'USDJPY.a': 0.0005196154845412693,
 'NZDUSD.a': -0.0022458744263254005,
 'EURUSD.a': -0.0032070555221488384}

In [1046]:
-0.05 / 1.35

-0.037037037037037035

In [6]:
warnings.filterwarnings("ignore")
mt5.initialize()
# Replace following with your MT5 Account Login
account = 926350
password = 'ADByx_74hJ' 
server = 'BlackBullMarkets-FPDemo'

### Monte Carlo Strat 

In [1043]:
account_value = 12500
weights = scale_weights(weights)
monetary_allocation = {pair: (weight / 100) * account_value for pair, weight in weights.items()}

In [1044]:
mc_lot_sizes = calculate_lot_sizes(monetary_allocation, latest_prices, 'micro')

In [1045]:
eer_os(last_update, mc_lot_sizes, 'MC_eer')

Looping through AUDUSD
Buying AUDUSD
Looping through USDCAD
Looping through USDCHF
Looping through EURUSD
Selling EURUSD
Looping through GBPUSD
Buying GBPUSD
Looping through NZDUSD
Selling NZDUSD
Looping through USDJPY
Selling USDJPY


### Simple Strat

In [614]:
total = 0 
for val in lot_sizes.values():
    total += val

In [884]:
sim_sizes = {}

for sym in lot_sizes.keys():
    sim_sizes[sym] = float(round(total/7, 2) - 0.03)
sim_sizes

{'USDCAD.a_close': 0.020000000000000004,
 'USDCHF.a_close': 0.020000000000000004,
 'EURUSD.a_close': 0.020000000000000004,
 'AUDUSD.a_close': 0.020000000000000004,
 'GBPUSD.a_close': 0.020000000000000004,
 'NZDUSD.a_close': 0.020000000000000004,
 'USDJPY.a_close': 0.020000000000000004}

In [403]:
eer_os(last_update, sim_sizes)

Looping through USDCAD
Selling USDCAD
Looping through USDCHF
Looping through EURUSD
Buying EURUSD
Looping through AUDUSD
Buying AUDUSD
Looping through GBPUSD
Buying GBPUSD
Looping through NZDUSD
Selling NZDUSD
Looping through USDJPY
Selling USDJPY


### COT Strat

In [973]:
for pair, direction in cot_positions.items():
    if direction > 0:
        print(f"Going long on {pair[:6]}")
        # send_order(pair, 'buy', 0.02, 'cot_rep')
    else:
        print(f"Going short on {pair}")
        # send_order(pair[:6], 'sell', 0.02, 'cot_rep')

Going long on USDCHF
Going long on GBPUSD
Going short on USDJPY.a
Going long on EURJPY
Going long on EURUSD
Going long on NZDUSD
Going long on AUDUSD


### Risk Management

MAX DD PER DAY FROM HIGHEST ACC BALANCE (Including floating PNL) - 3% Models to deal with drawdown limits:

Daily forecast of expected returns for the day and adjust accordingly.
Constant PNL monitoring algorithm to:
Take profits when floating PNL exceeds a certain threshold. For example, if current PNL is 7%, take profit on 5% and keep 2% floating.
For the drawdown side, continually hedge / adjust positions based on monte-carlo simulations at -1%, -1.5% and -2%. Close all positions at -2.25% floating, resume positions the next day.

Need to:
- Record beginning equity at 00:00 gmt+2 to calculate floating position pnl relative to 'beginning equity'

In [1023]:
from datetime import datetime, timezone, timedelta

In [1037]:
-0.03 / 1.35

-0.02222222222222222

In [1024]:
def close_position(position):

    tick = mt5.symbol_info_tick(position.symbol)

    request = {
        "action" : mt5.TRADE_ACTION_DEAL,
        "position": position.ticket,
        "symbol": position.symbol,
        "volume": position.volume,
        "type": mt5.ORDER_TYPE_BUY if position.type == 1 else mt5.ORDER_TYPE_SELL,
        "price": tick.ask if position.type == 1 else tick.bid,
        "deviation": 20,
        "magic": 100,
        "comment": 'Regres Close',
        'type_time': mt5.ORDER_TIME_GTC,
        'type_filling':mt5.ORDER_FILLING_IOC,

        }
    result = mt5.order_send(request)
    
def close_all():
    close_positions = []
    open_positions = mt5.positions_get()
    open_positions
    for i in open_positions:
        close_positions.append(i)
        
    for pos in close_positions:
        close_position(pos)
        
class ProfitLossControl:
    
    def __init__(self, max_daily_dd, profit_threshold, hedging_threshold, starting_equity):
        self.max_daily_dd = max_daily_dd
        self.profit_threshold = profit_threshold
        self.hedging_threshold = hedging_threshold
        self.starting_equity = starting_equity
    
    def start_equity(self):

        gmt_plus_2 = pytz.timezone('EET')  # Eastern European Time is typically GMT+2

        # Get the current time in UTC and convert it to GMT+2
        now_utc = datetime.now(timezone.utc)
        time = now_utc.astimezone(gmt_plus_2)
        now_gmt_plus_2 = now_utc.astimezone(gmt_plus_2)

        # Create a datetime object for the reset time (00:00:01) on the same day in GMT+2
        reset_time_today = now_gmt_plus_2.replace(hour=0, minute=0)

        if now_gmt_plus_2 != reset_time_today:
            info = mt5.account_info()
            start_equity = info.equity
            print(f"Starting equity is {start_equity}")

            return start_equity

        else:
            print(f"{countdown} remaining until equity reset")

    def day_pnl(self, start_equity): # Return current days equity drawdow / profit as a percentage from the start of the day. 
        
        # Define the GMT+2 timezone
        gmt_plus_2 = pytz.timezone('EET')  # Eastern European Time is typically GMT+2

        # Get the current time in UTC and convert it to GMT+2
        now_utc = datetime.now(timezone.utc)
        time = now_utc.astimezone(gmt_plus_2)
        now_gmt_plus_2 = now_utc.astimezone(gmt_plus_2)

        # Create a datetime object for the reset time (00:00:01) on the same day in GMT+2
        reset_time_today = now_gmt_plus_2.replace(hour=0, minute=0, second=1, microsecond=0)

        # Calculate the countdown to the reset time
        if now_gmt_plus_2 >= reset_time_today:
            # If the current time is after the reset time, calculate the countdown to the next day's reset time
            reset_time_tomorrow = reset_time_today + timedelta(days=1)
            countdown = reset_time_tomorrow - now_gmt_plus_2
        else:
            # If the current time is before today's reset time, calculate the countdown to today's reset time
            countdown = reset_time_today - now_gmt_plus_2
        
        account_info = mt5.account_info()
        equity = account_info.balance + account_info.profit
        
        day_pnl = equity - start_equity

        print(day_pnl)
        print(f"Time until next reset: {countdown}")
        
        return int(day_pnl)
        
    def daily_dd_control(self, day_pnl): 
        
        if day_pnl <= (self.max_daily_dd / 1.35):
            print('Closing all positions for the day')
            close_all()
        else:
            print(f'Threshold not reached. Current day pnl percent: {day_pnl}')
        
obj = ProfitLossControl(-0.03, 0.03, 0.00125, 6000)

In [1034]:
start_equity = obj.start_equity() # script will be running at the start of every 9am day // 00:00 gmt +2

Starting equity is 5900.91


In [1035]:
start_equity

5900.91

In [1036]:
day_pnl = obj.day_pnl(start_equity)
day_pnl

-0.07999999999992724
Time until next reset: 11:40:56.570616


0

# FTMO Accounts

In [265]:
account_val = 0
monetary_allocation = {pair: (weight / 100) * account_value for pair, weight in weights.items()}